# LunAPI : projects

Links to notebooks in this repository: [Index](./00_overview.ipynb) | [Individuals](./01_indivs.ipynb) | [Projects](./02_projects.ipynb) | [Staging](./03_staging.ipynb) | [Models](./04_models.ipynb) | [Advanced](./98_advanced.ipynb) | [Reference](./99_reference.ipynb)

(This page is under development)

In [1]:
import lunapi as lp
proj = lp.proj()

initiated lunapi v0.0.4 <lunapi.lunapi0.luna object at 0xffffb8a3cc70> 

enabling console outputs


### Working with projects

A _project_ is defined by a list of individual samples (EDFs/annotations).  We can then attach a sample list, either from a file (`read_sample_list()`) or build one on-the-fly.  This function returns the number of observations (EDF/annotation pairs), which is 3 in this case, but stores the information on the samples internally.

We can then attach a sample list, either from a file (`read_sample_list()`) or build one on-the-fly.  This function returns the number of observations (EDF/annotation pairs), but stores the information on the samples internally.

In [2]:
proj.build( [ '/tutorial/' , '-ext=-profusion.xml' ] ) 

3

You can see the information of the attached sample list as follows:

In [3]:
proj.sample_list()

[('learn-nsrr01',
  '/tutorial/edfs/learn-nsrr01.edf',
  {'/tutorial/edfs/learn-nsrr01-profusion.xml'}),
 ('learn-nsrr02',
  '/tutorial/edfs/learn-nsrr02.edf',
  {'/tutorial/edfs/learn-nsrr02-profusion.xml'}),
 ('learn-nsrr03',
  '/tutorial/edfs/learn-nsrr03.edf',
  {'/tutorial/edfs/learn-nsrr03-profusion.xml'})]

### Reading a sample-list from a file

If you have a pre-existing sample list (e.g. manually created, or generated by Luna's `--build` option), you can alternatively attach it with `sample_list( <filename> )`:

In [4]:
proj.sample_list( '/tutorial/s.lst' )

read 3 individuals from /tutorial/s.lst


In [5]:
proj.var( 'path' , '/tutorial/' )

In [6]:
proj.sample_list('/tutorial/s.lst')

read 3 individuals from /tutorial/s.lst


In [7]:
proj.sample_list()

[('nsrr01',
  '/tutorial/edfs/learn-nsrr01.edf',
  {'/tutorial/edfs/learn-nsrr01-profusion.xml'}),
 ('nsrr02',
  '/tutorial/edfs/learn-nsrr02.edf',
  {'/tutorial/edfs/learn-nsrr02-profusion.xml'}),
 ('nsrr03',
  '/tutorial/edfs/learn-nsrr03.edf',
  {'/tutorial/edfs/learn-nsrr03-profusion.xml'})]

This is similar to invoking Luna on the command line (if from this directory) as follows:

## Running a command across a project

To apply a Luna command to all individuals in a sample list use `eval()`:

In [8]:
res = proj.eval( 'STATS sig=EEG' )

___________________________________________________________________
Processing: nsrr01 | /tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG
 processing EEG ...
___________________________________________________________________
Processing: nsrr02 | /tutorial/edfs/learn-nsrr02.edf
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG
 processing EEG ...
______________________________

In [9]:
lp.show( res )

STATS: CH


,ID,CH,KURT,MAX,MEAN,MIN,P01,P02,P05,P10,...,P60,P70,P80,P90,P95,P98,P99,RMS,SD,SKEW
0,nsrr01,EEG,4.657487,125.0,-0.301199,-124.019608,-124.019608,-124.019608,-70.098039,-24.019608,...,2.45098,4.411765,8.333333,23.039216,72.058824,125.0,125.0,37.801351,37.800154,0.086212
1,nsrr02,EEG,3.546719,125.0,-0.370447,-124.019608,-124.019608,-124.019608,-78.921569,-33.823529,...,4.411765,8.333333,14.215686,30.882353,75.980392,125.0,125.0,41.044234,41.042566,-0.019268
2,nsrr03,EEG,1.345607,125.0,-0.11223,-124.019608,-124.019608,-124.019608,-124.019608,-73.039216,...,3.431373,7.352941,15.196078,70.098039,125.0,125.0,125.0,54.2708,54.270689,-0.011078


In [10]:
proj.strata()

,Command,Stata
0,STATS,CH


In [11]:
df = proj.table( 'STATS' , 'CH' )

In [12]:
df[ ['MEAN','KURT','SKEW' ] ].to_numpy(dtype=float)

array([[-0.3011987 ,  4.65748664,  0.08621198],
       [-0.37044745,  3.54671873, -0.01926794],
       [-0.11222989,  1.34560684, -0.01107757]])

In [13]:
df[ ['ID','KURT'] ] 

,ID,KURT
0,nsrr01,4.657487
1,nsrr02,3.546719
2,nsrr03,1.345607


## Adding individual-specific variables

In [14]:
%%sh
#echo -ne 'ID\ts1\n' > s1.txt
#echo -ne 'nsrr01\tEEG\n' >> s1.txt
#echo -ne 'nsrr02\tEMG\n' >> s1.txt
#echo -ne 'nsrr03\tECG\n' >> s1.txt
cat s1.txt

ID	s1
nsrr01	EEG
nsrr02	EMG
nsrr03	ECG


In [15]:
proj.var( 'vars' , 's1.txt' )

In [16]:
res = proj.eval( 'STATS sig=${s1}' ) 

___________________________________________________________________
Processing: nsrr01 | /tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EEG
 processing EEG ...
___________________________________________________________________
Processing: nsrr02 | /tutorial/edfs/learn-nsrr02.edf
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: STATS
   options: sig=EMG
 processing EMG ...
______________________________

In [17]:
proj.strata()

,Command,Stata
0,STATS,CH


In [18]:
proj.table( 'STATS' , 'CH' )

,ID,CH,KURT,MAX,MEAN,MIN,P01,P02,P05,P10,...,P60,P70,P80,P90,P95,P98,P99,RMS,SD,SKEW
0,nsrr03,ECG,11.165007,1.25,0.003571,-1.240196,-1.240196,-1.240196,-0.259804,-0.142157,...,0.02451,0.034314,0.063725,0.142157,0.259804,1.25,1.25,0.30003,0.300009,-0.094024
1,nsrr01,EEG,4.657487,125.0,-0.301199,-124.019608,-124.019608,-124.019608,-70.098039,-24.019608,...,2.45098,4.411765,8.333333,23.039216,72.058824,125.0,125.0,37.801351,37.800154,0.086212
2,nsrr02,EMG,-0.926233,31.5,-0.609767,-31.252941,-31.252941,-31.252941,-31.252941,-31.252941,...,1.111765,7.041176,20.629412,31.5,31.5,31.5,31.5,19.706464,19.69703,0.185887
